In [9]:
import os
import requests
from dbfread import DBF
import math
import random

# Parameters
dbf_file = "seamounts20.dbf"  # Path to the .dbf file
output_folder = "seamounts_galore"  # Folder to save downloaded images
tile_pixels = 800  # Width and height of the image in pixels
tile_width_deg = 1 / 3  # 20' x 20' = 1/3° x 1/3° (in degrees)
num_samples = 20  # Number of random tiles to download
api_base_url = "https://www.gmrt.org/services/ImageServer"

# Ensure output folder exists
os.makedirs(output_folder, exist_ok=True)

# Load .dbf file
try:
    dbf_data = list(DBF(dbf_file))  # Load all records into a list
    print(f"Loaded {len(dbf_data)} records from {dbf_file}.")
except Exception as e:
    print(f"Error loading .dbf file: {e}")
    exit()

# Shuffle the dataset to ensure randomness
random.shuffle(dbf_data)

# Randomly sample records
sampled_records = random.sample(dbf_data, min(num_samples, len(dbf_data)))

# Process each sampled record
for record in sampled_records:
    try:
        # Extract data from the record
        file_name = str(record["PEAKID"])  # Use PEAKID as the file name
        center_lon = float(record["LONG"])  # Longitude from LONG
        center_lat = float(record["LAT"])  # Latitude from LAT

        # Calculate tile bounds (20' x 20')
        minlatitude = center_lat - (tile_width_deg / 2)
        maxlatitude = center_lat + (tile_width_deg / 2)
        minlongitude = center_lon - (tile_width_deg / 2)
        maxlongitude = center_lon + (tile_width_deg / 2)

        # Prepare API parameters
        params = {
            "minlatitude": minlatitude,
            "maxlatitude": maxlatitude,
            "minlongitude": minlongitude,
            "maxlongitude": maxlongitude,
            "width": tile_pixels,
            "mask": "false",
            "download": "true",
        }

        # Construct file name
        output_file = os.path.join(output_folder, f"{file_name}.png")

        # Download the image
        print(f"Downloading image for PEAKID {file_name} at {center_lat}, {center_lon}...")
        response = requests.get(api_base_url, params=params)

        if response.status_code == 200:
            # Save the image
            with open(output_file, "wb") as file:
                file.write(response.content)
            print(f"Saved: {output_file}")
        else:
            print(f"Failed to download for PEAKID {file_name} - HTTP {response.status_code}")

    except Exception as e:
        print(f"Error processing record {record}: {e}")

print("Download complete.")


Loaded 19 records from seamounts20.dbf.
Saved: seamounts_galore/26298.png
Saved: seamounts_galore/26297.png
Saved: seamounts_galore/27045.png
Saved: seamounts_galore/27393.png
Saved: seamounts_galore/26158.png
Saved: seamounts_galore/27347.png
Saved: seamounts_galore/27112.png
Saved: seamounts_galore/26000.png
Saved: seamounts_galore/26157.png


KeyboardInterrupt: 